In [15]:
import argparse
from elasticsearch import Elasticsearch
from os import listdir
from numpy import RankWarning
import pandas as pd
from time import sleep

# #PARSE ARGUMENTS
# parser = argparse.ArgumentParser()
# parser.add_argument("-i", "--input-dir")
# parser.add_argument("-o", "--output-dir")
# args = parser.parse_args()
# args = vars(args)

# #HANDLE INPUT
# input_dir = args['input_dir']
# output_dir = args['output_dir']

# sleep(60)

es = Elasticsearch(hosts="localhost")

while not es.ping():
    sleep(10)
    print("WAITING...")
    es = Elasticsearch(hosts="http://irlab_elastic_1:9200", timeout=300)
print("done waiting")

###############
# make dictionary with all topics
import xml.etree.ElementTree as ET
tree = ET.parse('topics.xml')
# tree = ET.parse('topics_selected.xml')
topics = tree.findall('topic')

# Dictionary with all 50 topics
topicsDic = {}

for topic in topics:
    title = topic.find('title').text
    number = topic.find('number').text
    topicsDic[number] = title

num_results = 10

## first run: sentiment+OCR ###

def search_refined_OCR_prototype(query, num_results):
    body_positive = {
        "from":0,
        "size":num_results,
        "query": {
            "bool":{
                "should":[
                    {"match": { "document_text":query}},
                    {"match": {"ocr_text":{"query":query, "boost":5}}}  # ocr_text sollte einen natürlichen boost besitzen, da die texte viel kürzer sind. Vielleicht muss dieser auch abgeschwächt werden?
                
                ],
                "filter":[
                    {"range": {"sentiment": {"gt": 0}}}
                ]
            }   
        }
    }
    res_positive = es.search(index="final_boromir_index", body=body_positive)

    body_negative = {
        "from":0,
        "size":num_results,
        "query": {
            "bool":{
                "should":[
                    {"match": { "document_text":query}},
                    {"match": {"ocr_text":{"query":query, "boost":5}}}  # ocr_text sollte einen natürlichen boost besitzen, da die texte viel kürzer sind. Vielleicht muss dieser auch abgeschwächt werden?
                

                ],
                "filter":[
                    {"range": {"sentiment": {"lt": 0}}}
                ]            
            }   
        }
    }
    res_negative = es.search(index="final_boromir_index", body=body_negative)

    # get ID's from retrieved documents
    resultPos = []
    results_positive = res_positive.get('hits').get('hits')

    for doc in results_positive:
        id = doc.get('_id')
        score = doc.get('_score')
        resultPos.append([id, score])

    resultNeg = []
    results_negative = res_negative.get('hits').get('hits')

    for doc in results_negative:
        id = doc.get('_id')
        score = doc.get('_score')
        resultNeg.append([id, score])

    return resultPos, resultNeg

resultList = []
for topic in topicsDic:
    query = topicsDic.get(topic)
    result_ids_pro, result_ids_con = search_refined_OCR_prototype(query, num_results)
    i = 1
    for each in result_ids_pro:
        resultList.append([topic, "PRO", each[0], i, each[1], "Boromir1"])
        i += 1
    i = 1
    for each in result_ids_con:
        resultList.append([topic, "CON", each[0], i, each[1], "Boromir1"])
        i += 1

resultdf = pd.DataFrame(resultList, columns = ['topicID','stance','pageID','rank','score','Method'])
with open(f'run.txt', 'a+') as f:
    resultdf[['topicID','stance','pageID','rank','score','Method']].to_csv(f, sep=' ', header=False, index=False)

print(resultdf)

done waiting


C:\Users\thilo\AppData\Local\Temp/ipykernel_24120/365715216.py:65: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res_positive = es.search(index="final_boromir_index", body=body_positive)
C:\Users\thilo\AppData\Local\Temp/ipykernel_24120/365715216.py:84: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res_negative = es.search(index="final_boromir_index", body=body_negative)


    topicID stance             pageID  rank      score    Method
0         1    PRO  I2dda022878455420     1  85.991165  Boromir1
1         1    PRO  I211441c0453def35     2  83.126816  Boromir1
2         1    PRO  I9e60517c2520ebe5     3  79.457180  Boromir1
3         1    PRO  Ic21db5313b5a4d9c     4  68.820786  Boromir1
4         1    PRO  I72f92221cbde52e1     5  68.136475  Boromir1
..      ...    ...                ...   ...        ...       ...
995      50    CON  Ia545769f4e369b40     6  76.133160  Boromir1
996      50    CON  Ie43ab05b64bdb708     7  72.860340  Boromir1
997      50    CON  I03e2e1d48ab74152     8  61.380455  Boromir1
998      50    CON  I7f81e8a4a867b336     9  53.233340  Boromir1
999      50    CON  I09b98cea5f14f020    10  52.114150  Boromir1

[1000 rows x 6 columns]


In [5]:
import argparse
from elasticsearch import Elasticsearch
from os import listdir
import pandas as pd
from time import sleep

# #PARSE ARGUMENTS
# parser = argparse.ArgumentParser()
# parser.add_argument("-i", "--input-dir")
# parser.add_argument("-o", "--output-dir")
# args = parser.parse_args()
# args = vars(args)

# #HANDLE INPUT
# input_dir = args['input_dir']
# output_dir = args['output_dir']

# sleep(60)

es = Elasticsearch(hosts="localhost")

while not es.ping():
    sleep(10)
    print("WAITING...")
    es = Elasticsearch(hosts="localhost", timeout=300)
print("done waiting")

###############
# make dictionary with all topics
import xml.etree.ElementTree as ET
tree = ET.parse('topics.xml')
# tree = ET.parse('topics_selected.xml')
topics = tree.findall('topic')

# Dictionary with all 50 topics
topicsDic = {}

for topic in topics:
    title = topic.find('title').text
    number = topic.find('number').text
    topicsDic[number] = title

num_results = 10

## second run: sentiment+ImageClustering ###

def search_refined_Cluster(query, num_results):
    body_positive = {
        "from":0,
        "size":num_results,
        "query": {
        "function_score": {
        "query": {
            "bool":{
                "should":[
                    {"match": { "document_text":{"query":query}}},
                ],

                "filter":[
                    {"range": {"sentiment": {"gt": 0}}}
                ]
            }   
        },
        #"boost": "5", 
        "functions": [
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "0" } },"weight": 5.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "1" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "2" } },"weight": 3.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "3" } },"weight": 3.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "4" } },"weight": 4.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "5" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "6" } },"weight": 5.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "7" } },"weight": 3.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "8" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "9" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "10" } },"weight": 2.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "11" } },"weight": 5.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "12" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "13" } },"weight": 2.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "999" } },"weight": 0.0}
        ],      
        "max_boost": 5.0,
        "score_mode": "max",
        "boost_mode": "multiply",
        "min_score": 0.0
        }
        }
    }

    res_positive = es.search(index="final_boromir_index", body=body_positive)

    body_negative = {
        "from":0,
        "size":num_results,
        "query": {
        "function_score": {
        "query": {
            "bool":{
                "should":[
                    {"match": { "document_text":{"query":query}}},
                ],

                "filter":[
                    {"range": {"sentiment": {"lt": 0}}}
                ]
            }   
        },
        #"boost": "5", 
        "functions": [
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "0" } },"weight": 5.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "1" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "2" } },"weight": 3.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "3" } },"weight": 3.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "4" } },"weight": 4.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "5" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "6" } },"weight": 5.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "7" } },"weight": 3.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "8" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "9" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "10" } },"weight": 2.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "11" } },"weight": 5.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "12" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "13" } },"weight": 2.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "999" } },"weight": 0.0}
        ],      
        "max_boost": 5.0,
        "score_mode": "max",
        "boost_mode": "multiply",
        "min_score": 0.0
        }
        }
    }

    res_negative = es.search(index="final_boromir_index", body=body_negative)

    # get ID's from retrieved documents
    result_positive_ids = []
    results_positive = res_positive.get('hits').get('hits')

    for doc in results_positive:
        id = doc.get('_id')
        result_positive_ids.append(id)

    result_negative_ids = []
    results_negative = res_negative.get('hits').get('hits')

    for doc in results_negative:
        id = doc.get('_id')
        result_negative_ids.append(id)

    return result_positive_ids, result_negative_ids

resultList = []
for topic in topicsDic:
    query = topicsDic.get(topic)
    result_ids_pro, result_ids_con = search_refined_Cluster(query, num_results)
    for each in result_ids_pro:
        resultList.append([topic, "PRO", each, 0, 0.0, "Boromir2"])
    for each in result_ids_con:
        resultList.append([topic, "CON", each, 0, 0.0, "Boromir2"])

resultdf = pd.DataFrame(resultList, columns = ['topicID','stance','pageID','rank','score','Method'])
with open(f'run.txt', 'a+') as f:
    resultdf[['topicID','stance','pageID','rank','score','Method']].to_csv(f, sep=' ', header=False, index=False)

print(resultdf)

C:\Users\thilo\AppData\Local\Programs\Python\Python39\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
C:\Users\thilo\AppData\Local\Temp/ipykernel_24120/4019254249.py:90: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res_positive = es.search(index="final_boromir_index", body=body_positive)
C:\Users\thilo\AppData\Local\Temp/ipykernel_24120/4019254249.py:134: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead 

done waiting
    topicID stance             pageID  rank  score    Method
0         1    PRO  I4592105915bb12fd     0    0.0  Boromir2
1         1    PRO  I9bbcdf9b97e28232     0    0.0  Boromir2
2         1    PRO  I912c93b9a9c4e7bb     0    0.0  Boromir2
3         1    PRO  I6f276d3886624d52     0    0.0  Boromir2
4         1    PRO  Idf7d5a8242c3c6ff     0    0.0  Boromir2
..      ...    ...                ...   ...    ...       ...
995      50    CON  I77b5c4ed96425a65     0    0.0  Boromir2
996      50    CON  I011b22837ad2dead     0    0.0  Boromir2
997      50    CON  I569956ec000d89b2     0    0.0  Boromir2
998      50    CON  Ifa19e28004aaceba     0    0.0  Boromir2
999      50    CON  Ib2fb69fd1ac42329     0    0.0  Boromir2

[1000 rows x 6 columns]


In [ ]:
import argparse
from elasticsearch import Elasticsearch
from os import listdir
import pandas as pd
from time import sleep

# #PARSE ARGUMENTS
# parser = argparse.ArgumentParser()
# parser.add_argument("-i", "--input-dir")
# parser.add_argument("-o", "--output-dir")
# args = parser.parse_args()
# args = vars(args)

# #HANDLE INPUT
# input_dir = args['input_dir']
# output_dir = args['output_dir']

es = Elasticsearch(hosts="localhost")

while not es.ping():
    sleep(10)
    print("WAITING...")
    es = Elasticsearch(hosts="localhost", timeout=300)
print("done waiting")

###############
# make dictionary with all topics
import xml.etree.ElementTree as ET
tree = ET.parse('topics.xml')
# tree = ET.parse('topics_selected.xml')
topics = tree.findall('topic')

# Dictionary with all 50 topics
topicsDic = {}

for topic in topics:
    title = topic.find('title').text
    number = topic.find('number').text
    topicsDic[number] = title

num_results = 10

## third run: sentiment+OCR+ImageClustering###
def search_refined_OCR_Clustering_prototype(query, num_results):
    body_positive = {
        "from":0,
        "size":num_results,
        "query": {
        "function_score": {
        "query": {
            "bool":{
                "should":[
                    {"match": { "document_text":{"query":query}}},
                    {"match": {"ocr_text":{"query":query, "boost":5}}}
                ],

                "filter":[
                    {"range": {"sentiment": {"gt": 0}}}
                ]
            }   
        },
        #"boost": "5", 
        "functions": [
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "0" } },"weight": 5.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "1" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "2" } },"weight": 3.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "3" } },"weight": 3.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "4" } },"weight": 4.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "5" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "6" } },"weight": 5.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "7" } },"weight": 3.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "8" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "9" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "10" } },"weight": 2.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "11" } },"weight": 5.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "12" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "13" } },"weight": 2.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "999" } },"weight": 0.0}
        ],      
        "max_boost": 5.0,
        "score_mode": "max",
        "boost_mode": "multiply",
        "min_score": 0.0
        }
        }
    }
    res_positive = es.search(index="final_boromir_index", body=body_positive)

    body_negative = {
        "from":0,
        "size":num_results,
        "query": {
        "function_score": {
        "query": {
            "bool":{
                "should":[
                    {"match": { "document_text":{"query":query}}},
                    {"match": {"ocr_text":{"query":query, "boost":5}}}
                ],

                "filter":[
                    {"range": {"sentiment": {"lt": 0}}}
                ]
            }   
        },
        #"boost": "5", 
        "functions": [
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "0" } },"weight": 5.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "1" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "2" } },"weight": 3.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "3" } },"weight": 3.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "4" } },"weight": 4.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "5" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "6" } },"weight": 5.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "7" } },"weight": 3.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "8" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "9" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "10" } },"weight": 2.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "11" } },"weight": 5.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "12" } },"weight": 1.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "13" } },"weight": 2.0},
            {"filter": { "match": { "cluster_when_14_clusters_in_10dim": "999" } },"weight": 0.0}
        ],      
        "max_boost": 5.0,
        "score_mode": "max",
        "boost_mode": "multiply",
        "min_score": 0.0
        }
        }
    }
    res_negative = es.search(index="final_boromir_index", body=body_negative)

    # get ID's from retrieved documents
    result_positive_ids = []
    results_positive = res_positive.get('hits').get('hits')

    for doc in results_positive:
        id = doc.get('_id')
        result_positive_ids.append(id)

    result_negative_ids = []
    results_negative = res_negative.get('hits').get('hits')

    for doc in results_negative:
        id = doc.get('_id')
        result_negative_ids.append(id)

    return result_positive_ids, result_negative_ids

resultList = []
for topic in topicsDic:
    query = topicsDic.get(topic)
    result_ids_pro, result_ids_con = search_refined_OCR_Clustering_prototype(query, num_results)
    for each in result_ids_pro:
        resultList.append([topic, "PRO", each, 0, 0.0, "Boromir3"])
    for each in result_ids_con:
        resultList.append([topic, "CON", each, 0, 0.0, "Boromir3"])

resultdf = pd.DataFrame(resultList, columns = ['topicID','stance','pageID','rank','score','Method'])
with open(f'run.txt', 'a+') as f:
    resultdf[['topicID','stance','pageID','rank','score','Method']].to_csv(f, sep=' ', header=False, index=False)

print(resultdf)


In [7]:
results_negative

NameError: name 'results_negative' is not defined